In [17]:
# ---------------------------------------------------------------------
# 0. Clean environment and reload local package
# ---------------------------------------------------------------------
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath("."))

from energy.preprocess import read_data, expiry_calendar, drop_dupes
from energy.strategies import rolling_pnl, carry
from energy.analytics import metrics

import pandas as pd
import numpy as np

print("✅ Libraries imported successfully.")

# ---------------------------------------------------------------------
# 1. Load WTI prices and expiry calendar using preprocess
# ---------------------------------------------------------------------

wti_path = "data/Commods.xlsx"
exp_path = "data/expiry_calendars.xlsx"

# Load standardized WTI data
wti_df = read_data(wti_path, sheet="WTI (class)")

# Load expiry calendar (ticker first, then optional path)
expiry_cal = expiry_calendar("CL", calendar_path=exp_path)


print(f"WTI shape: {wti_df.shape}")
print(f"Expiry count: {len(expiry_cal)}")
print(wti_df.head())
print(expiry_cal[:5])


# ---------------------------------------------------------------------
# 2. Generate rolling PnL
# ---------------------------------------------------------------------
rolled = rolling_pnl(
    prices=wti_df,
    expiry_calendar=expiry_cal,
    front_col="F1",
    next_col="F2",
    roll_window=6,
)

# ---------------------------------------------------------------------
# 3. Run Carry Strategy
# ---------------------------------------------------------------------
carry_df = carry(
    prices=wti_df,
    rolled_df=rolled,
    front_col="F1",
    end_col="F4",
    t_cost=0.01,
    epsilon=0.0,
)

# ---------------------------------------------------------------------
# 4. Compute Metrics
# ---------------------------------------------------------------------
metrics_out = metrics(df=carry_df, contracts=100, units=1000)
print("\n----- Carry Strategy Metrics -----")
print(metrics_out)

# ---------------------------------------------------------------------
# 5. Export diagnostic Excel to inspect carry series
# ---------------------------------------------------------------------
signal = np.sign(wti_df["F1"] - wti_df["F4"]).reindex(carry_df.index).fillna(0)
signal_lag = signal.shift(1).fillna(0)

debug_df = carry_df.copy()
debug_df["signal"] = signal
debug_df["signal_lag"] = signal_lag
debug_df["rolled_t_cost"] = rolled["t_cost"].reindex(carry_df.index)
debug_df["roll_flag"] = rolled["roll_day_flag"].reindex(carry_df.index)
debug_df["raw_daily_pnl"] = rolled["daily_pnl"].reindex(carry_df.index).squeeze()
debug_df["cum_net_pnl"] = debug_df["net_pnl"].cumsum()
debug_df["cum_raw_pnl"] = debug_df["raw_daily_pnl"].cumsum()

cols = [
    "signal",
    "signal_lag",
    "roll_flag",
    "daily_pnl",
    "t_cost",
    "rolled_t_cost",
    "net_pnl",
    "equity_line",
    "cum_net_pnl",
    "cum_raw_pnl",
]

debug_path = "carry_equity_debug.xlsx"
debug_df[cols].to_excel(debug_path, sheet_name="CarryDebug", index=True)
print(f"\n✅ Full carry equity diagnostics written to:\n   {os.path.abspath(debug_path)}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Libraries imported successfully.
WTI shape: (3524, 4)
Expiry count: 313
                F1      F2      F3      F4
Dates                                     
2010-01-04 81.5100 82.1200 82.6500 83.1200
2010-01-05 81.7700 82.4100 82.9900 83.5200
2010-01-06 83.1800 83.7500 84.3100 84.8600
2010-01-07 82.6600 83.1900 83.7500 84.2900
2010-01-08 82.7500 83.3000 83.8700 84.4700
DatetimeIndex(['2010-01-20', '2010-02-22', '2010-03-22', '2010-04-20',
               '2010-05-20'],
              dtype='datetime64[ns]', freq=None)

--- Debug cost reconciliation ---
t_cost sum inside df: 4.9800
Total cost (contracts×units): 498,000.00
Total PnL (sum of net_pnl): 9,545,000.00

----- Carry Strategy Metrics -----
Total PnL        9,545,000.0000
Total Cost         498,000.0000
APL/unit                 6.8265
Std Dev (ann.)          24.7473
Sharpe                   0.2800
Drawdown                -0.2913
RoD          